In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle
import random
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import pandas as pd
from tqdm import tqdm
import pickle
from sklearn.model_selection import GridSearchCV

In [2]:
dir = 'Tensorflow\workspace\images\\train 9'
label_map = pd.read_csv('baybayin character 8.csv')

In [3]:
data = []

for index, row in label_map.iterrows():
    category = row['label']
    label = row['id']
    path = os.path.join(dir, category)

    for img in os.listdir(path):
        imgpath = os.path.join(path, img)
        a_img = cv2.imread(imgpath, 0)

        # Apply Gaussian blur to the image
        a_img = cv2.GaussianBlur(a_img, (5, 5), cv2.BORDER_DEFAULT)

        # Apply Otsu thresholding to the image
        _, a_img = cv2.threshold(a_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Apply edge detection to the image
        #a_img = cv2.Canny(a_img, 100, 200)

        # Rotate the image randomly between -15 to 15 degrees
        rows, cols = a_img.shape[:2]
        angle = np.random.randint(-15, 15)
        rotation_matrix = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
        a_img = cv2.warpAffine(a_img, rotation_matrix, (cols, rows))

        a_img = cv2.resize(a_img, (50, 50))
        image = np.array(a_img).flatten()

        data.append([image, label])

print(len(data))

9900


In [4]:
random.shuffle(data)
features = []
labels = []

for feature,label in data:
    features.append(feature)
    labels.append(label)

In [5]:
xtrain, xtest, ytrain, ytest = train_test_split(features, labels, test_size=0.20)

In [6]:
checkpoint_dir = "model_checkpoints/"

if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

In [7]:
# Train the model and save checkpoints
model = SVC(C= 25, kernel='rbf', gamma = 1e-08, probability=True)

progress = tqdm(range(0, 100, 10), desc="Training", unit="epoch")
for i in progress:
    # Fit the model for each epoch
    model.fit(xtrain, ytrain)
    # Save the checkpoint
    checkpoint_file = os.path.join(checkpoint_dir, f"model_{i+1}.sav")
    pickle.dump(model, open(checkpoint_file, "wb"))
    # Calculate and display the accuracy on the test set
    accuracy = model.score(xtest, ytest)
    progress.set_postfix({"accuracy": accuracy})

Training: 100%|████████████████████████████████████████████████████| 10/10 [19:50<00:00, 119.06s/epoch, accuracy=0.978]


In [8]:
# Save the final trained model to a file.
final_model_file = "svm_baybayin_rbf_confusion_v3.sav"
pickle.dump(model, open(final_model_file, 'wb'))

In [9]:
accuracy = model.score(xtest, ytest)
print('accuracy', accuracy)

accuracy 0.9777777777777777


In [7]:
from sklearn.model_selection import GridSearchCV

parameters = {'C': [0.1, 1, 10], 'gamma': [0.1, 0.01, 0.001]}
grid_search = GridSearchCV(SVC(kernel='poly'), parameters)
grid_search.fit(xtrain, ytrain)

best_params = grid_search.best_params_
print(best_params)

{'C': 0.1, 'gamma': 0.1}


In [8]:
parameters = {'C': [25, 30, 20, 40, 35], 'gamma': [1e-09, 1e-010, 1e-07, 1e-08]}
grid_search = GridSearchCV(SVC(kernel='rbf'), parameters)
grid_search.fit(xtrain, ytrain)
 
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Accuracy:", best_score)

Best Parameters: {'C': 25, 'gamma': 1e-07}
Best Accuracy: 0.9804856115107914


In [12]:
parameters = {'C': [25, 30, 20, 1, 10], 'gamma': [0.01, 0.1, 0.0001]}
grid_search = GridSearchCV(SVC(kernel='linear'), parameters)
grid_search.fit(xtrain, ytrain)
 
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Accuracy:", best_score)

Best Parameters: {'C': 25, 'gamma': 0.01}
Best Accuracy: 0.9828333333333333
